<a href="https://colab.research.google.com/github/FaisalRashid9775/AI_Agents/blob/main/%5B11%5DGuardrils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Guardrails** :  

Guardrails are the safety check for a agent.It prevent agent from doing thing which is dangerous or harmful or costly.
For this your loop only need a function in which you have provided tripwire_triggered . If tripwire_triggered is true then loop raise an exception and prevent delivery. Same for both input and output.

input_guradrail and out_guardrail decorators convert a function into guardrails.

Use input_guradrail decorator for apply guardrail on user inputs.


Use output_guardrail decorator for apply guardrail on system output


In [ ]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.4/144.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 946.9/946.9 kB 19.6 MB/s eta 0:00:00


In [ ]:
from agents import Agent,Runner, AsyncOpenAI, OpenAIChatCompletionsModel, set_tracing_disabled, function_tool, input_guardrail, output_guardrail, GuardrailFunctionOutput,InputGuardrailTripwireTriggered,OutputGuardrailTripwireTriggered
from google.colab import userdata
from pydantic import BaseModel
class Users(BaseModel):
    reason : str
    trigger :bool
gemini_api_key = userdata.get('GOOGLE_API_KEY')
import nest_asyncio
nest_asyncio.apply()

client = AsyncOpenAI(api_key=gemini_api_key,
                     base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

lite_model  = OpenAIChatCompletionsModel(
    model='gemini-2.5-flash',
    openai_client=client
)
set_tracing_disabled(disabled=True)

weather_sanitizer = Agent(name='weather_sanitizer',instructions='Check wether the query is related to the weather',model=lite_model,output_type=Users )

@input_guardrail
def weather_input_guardrails(context,agent,input):
  res =Runner.run_sync(weather_sanitizer,input)    # Run weather_sanitizer agent which will check wether the query about weather or not.
  print(res.final_output)
  return GuardrailFunctionOutput(
      output_info='passed',
      tripwire_triggered=res.final_output.trigger is False
  )

@output_guardrail
def weather_output_guardrails(context,agent,output):
  return GuardrailFunctionOutput(
      output_info='passed',
      tripwire_triggered=False
  )
@function_tool()
def get_weather(city:str):
  print(city)
  return f'Weather in {city} is sunny and temperature is 25c and weather remains dry'

try :
  agent = Agent(
      name='agnet',
      instructions='Your are helpful agent Call tool',
      model=lite_model,
      input_guardrails=[weather_input_guardrails],
      output_guardrails=[weather_output_guardrails])
  response  = Runner.run_sync(agent,'How will the Agentic life')
  print(response.final_output)
except InputGuardrailTripwireTriggered as e:
  print('Alert : ',e)
except OutputGuardrailTripwireTriggered as e:
  print(e)

reason="The user is asking about 'Agentic life', which is not related to weather." trigger=False
Alert :  Guardrail InputGuardrail triggered tripwire
